# Config

In [1]:
DRY_RUN = False
N_SIMULTANEOUS_ARGO_JOBS = 20
N_SIMULATIONS = 100

# Setup

Build/deploy most recent container image

In [2]:
%%bash
docker build --platform linux/amd64 -t dberry-jetstream . 
docker tag dberry-jetstream gcr.io/moz-fx-data-experiments/dberry-jetstream
docker push gcr.io/moz-fx-data-experiments/dberry-jetstream

Using default tag: latest
The push refers to repository [gcr.io/moz-fx-data-experiments/dberry-jetstream]
a2e42e1359d5: Preparing
0e3aec3db855: Preparing
5f70bf18a086: Preparing
0ae468ad3315: Preparing
71aeb5b4699c: Preparing
95bfa4940c59: Preparing
8dc5d08744dd: Preparing
ff1f2a3b1a2d: Preparing
5b9e7e39f904: Preparing
182f3ce3bf27: Preparing
316e3949bffa: Preparing
e3f84a8cee1f: Preparing
48144a6f44ae: Preparing
26d5108b2cba: Preparing
89fda00479fc: Preparing
5b9e7e39f904: Waiting
182f3ce3bf27: Waiting
316e3949bffa: Waiting
e3f84a8cee1f: Waiting
48144a6f44ae: Waiting
26d5108b2cba: Waiting
89fda00479fc: Waiting
8dc5d08744dd: Waiting
95bfa4940c59: Waiting
ff1f2a3b1a2d: Waiting
5f70bf18a086: Layer already exists
71aeb5b4699c: Layer already exists
0ae468ad3315: Layer already exists
95bfa4940c59: Layer already exists
8dc5d08744dd: Layer already exists
ff1f2a3b1a2d: Layer already exists
5b9e7e39f904: Layer already exists
182f3ce3bf27: Layer already exists
316e3949bffa: Layer already exists

#1 [internal] load build definition from Dockerfile
#1 sha256:23428abc3552d980089f37e3529d936c9f6c7fe78a75b553e641e1118c38c79a
#1 transferring dockerfile: 37B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:628f5645742c0740dc4d82752bb82b94b2eb7df2d93cd15b6bbff5fafb5e57db
#2 transferring context: 34B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8
#3 sha256:edad251955f644c6004999f0af04035912392fa02db26821676452becbc715fb
#3 DONE 2.2s

#4 [1/7] FROM docker.io/library/python:3.8@sha256:53cb5152064a7e7b485ad42704ea63c5155b264c82e7f17de99d3aa28e4f8956
#4 sha256:2bcf9d7e1be5c851f4dd24e5b6cd59a3d8e90180dcc5894973a533b1625888d2
#4 DONE 0.0s

#5 [internal] load build context
#5 sha256:63492c8ae75977de819b51f8a1875e38ae043bc6feb6c631f39b0ad29d8ac7bd
#5 transferring context: 3.88MB 0.2s done
#5 DONE 0.2s

#7 [3/7] RUN python -m pip install --no-cache-dir --upgrade pip==22.0.3     && python -m pip install --no-cache-dir -r requirements.txt
#7 sha256:ad

Ensure Argo template is correct (in case 0_create_cache.sh was run immediately prior)

In [3]:
sed_str = f's/PARAM/{DRY_RUN}/g'

In [4]:
%%bash -s "$sed_str"
cp jetstream/workflows/run_template.yaml jetstream/workflows/run.yaml
sed -i '' -e $1 jetstream/workflows/run.yaml

Imports

In [5]:
import dask
import dask.array as da
from dask.distributed import Client
import subprocess

Set up dask client
- Set `threads_per_worker` to 1 to ensure that each worker only launches 1 job at a time
- Then set `n_workers` to the number of parallel Argo jobs you want to run

In [6]:
dask_client = Client(n_workers=N_SIMULTANEOUS_ARGO_JOBS, threads_per_worker=1)

Define a function which launches one simulation run using Argo

In [7]:
@dask.delayed
def run_simulation(simulation_iteration: int) -> bool:
    dataset = 'dberry_simulated_AA_tests_{:02d}'.format(simulation_iteration)
    
    drop_dataset_result = subprocess.run(['bq','rm', '-d', '-r', '-f', dataset])
    if drop_dataset_result.returncode != 0:
        print(f'Unable to delete dataset {dataset}')
        return (simulation_iteration, False)
        
    create_dataset_result = subprocess.run(['bq','mk', dataset], capture_output=True)
    if create_dataset_result.returncode != 0:
        print(f'Unable to create dataset {dataset}! {create_dataset_result.stdout} {create_dataset_result.stderr}')
        return (simulation_iteration, False)
        
    argo_run_result = subprocess.run([
        'jetstream',
        'run-argo',
        '--date','2022-02-22',
        '--project-id','moz-fx-data-experiments',
        '--dataset-id',f'{dataset}',
        '--bucket','dberry-simulated-aa-tests-temporary',
        '--experiment-slug', 'more-from-mozilla-96',
        '--cluster-id','jetstream',
        '--zone','us-central1-a'],
        capture_output=True
    )
    if argo_run_result.returncode != 0:
        print(f'Simulation {simulation_iteration} failed! {argo_run_result.stdout} {argo_run_result.stderr}')
        return (simulation_iteration, False)
        
    return (simulation_iteration, True)
        
    

Create a future for each of the number of simulations you want to run

In [8]:
sim_futures = [run_simulation(i) for i in range(N_SIMULATIONS)]

# Run Simulations
Note that the below cell will take a while, approximately $\left \lceil{\frac{\text{N_SIMULATIONS}}{\text{N_SIMULTANEOUS_ARGO_JOBS}}}\right \rceil \cdot 30$ minutes when not in dry run

In [9]:
%%time
res = dask.compute(*sim_futures)

CPU times: user 56min 50s, sys: 9min 32s, total: 1h 6min 23s
Wall time: 5h 46min 56s


In [10]:
res

((0, True),
 (1, True),
 (2, True),
 (3, True),
 (4, True),
 (5, True),
 (6, True),
 (7, True),
 (8, True),
 (9, True),
 (10, True),
 (11, True),
 (12, True),
 (13, True),
 (14, True),
 (15, True),
 (16, True),
 (17, True),
 (18, True),
 (19, True),
 (20, True),
 (21, True),
 (22, True),
 (23, True),
 (24, True),
 (25, True),
 (26, True),
 (27, False),
 (28, True),
 (29, True),
 (30, True),
 (31, True),
 (32, True),
 (33, True),
 (34, True),
 (35, True),
 (36, True),
 (37, True),
 (38, True),
 (39, True),
 (40, True),
 (41, True),
 (42, True),
 (43, True),
 (44, True),
 (45, True),
 (46, True),
 (47, True),
 (48, True),
 (49, True),
 (50, True),
 (51, True),
 (52, True),
 (53, True),
 (54, True),
 (55, True),
 (56, True),
 (57, True),
 (58, True),
 (59, True),
 (60, True),
 (61, True),
 (62, True),
 (63, True),
 (64, True),
 (65, True),
 (66, True),
 (67, True),
 (68, True),
 (69, True),
 (70, True),
 (71, True),
 (72, True),
 (73, True),
 (74, True),
 (75, True),
 (76, True),
 (77, Tr